# Data pipeline 구축하기 1  


Rag pipeline을 만들고 각 함수를 손쉽게 사용해보자

이번 포스팅에는 VectorDB를 정하고 거기에 pdf파일의 내용을 넣어본다.


## 0. Setting 

.env 파일을 만들어 API키들을 넣어준다. 

나중에 Ollema를 사용해서 API 없이 local에서 작동가능한 LLM을 사용해 보자.



```bash
OPENAI_API_KEY='sk-proj-5m5haMMQ0Sgkctb7Udixxxxx'


```

In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
# API KEY 정보로드
load_dotenv()

True

## 1. Data pipeline - VectorDB

In [2]:
# langchain_openai에서 ChatOpenAI(LLM)과 OpenAIEmbeddings(임베딩모델: text를 vector화하는 모델)을 load
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# PDF파일등 데이터를 Chroma형식의 vectorDB에 저장하고 리트리버가 수집한 데이터에 접근하기 위해 Chroma를 load
from langchain_community.vectorstores import Chroma

# 우리가 만든 config.py에서 모델등 옵션들을 수정
from utils.config import config

# 추가 데이터를 업로드하기 위해 file을 다큐먼트로 만들고 (convert_file_to_documents) 다큐먼트를 자르는 (split_document) 함수를 load합니다.
from utils.update import convert_file_to_documents

In [3]:
class Datapipeline:
    def __init__(self):

        # RAG가 접근할 vectorDB를 초기화합니다.
        self.vector_store = self.init_vectorDB()

    def init_vectorDB(self):
        """vectorDB 설정"""
        embeddings = OpenAIEmbeddings(model=config["embed_model"]["model_name"])  # VectorDB에 저장될 데이터를 임베딩할 모델을 선언합니다.
        vector_store = Chroma(
            persist_directory=config["chroma"]["persist_dir"],  # 기존에 vectordb가 있으면 해당 위치의 vectordb를 load하고 없으면 새로 생성합니다.
            embedding_function=embeddings,                      # 새롭게 데이터가 vectordb에 넣어질때 사용할 임베딩 방식을 정합니다, 저희는 위에서 선언한 embeddings를 사용합니다.
            collection_name = 'india',                          # india라는 이름을 정해줌으로써 나중에 vector store 관리 가능 
            collection_metadata = {'hnsw:space': 'cosine'},     # cosine 말고 l2 가 default / collection_metadata를 통해 유사도 검색에 사용될 공간('hnsw:space')을 'cosine'으로 지정하여, 코사인 유사도를 사용
        )
        return vector_store

    def update_vector_db(self, file, force=True, chunk_size=200):
        # txt파일, pdf파일, csv파일에서 파일 loader를 통해 내용과 소스를 documents에 넣습니다. 
        # force가 True인 경우 기존 VectorDB에 유사한 내용 유무와 상관없이 파일을 업로드 합니다. 
        # force가 False인 경우 기존 VectorDB에 유사한 내용이 있는지 확인하고 있으면 데이터를 업로드 하지 않습니다. 
        upload_documents = convert_file_to_documents(self.vector_store, file, force, chunk_size)

        if upload_documents:                                   # 잘라서 넣어줄 내용이 있다면 
            self.vector_store.add_documents(upload_documents)  # vectorDB에 넣어줍니다. 이때, vector_store 초기화 할때 정해준 embedding_function에 맞게 임베딩되어 들어갑니다. 
            print(f"Added {len(upload_documents)} new documents to the vector store")
            return True
        else:
            print("모두 유사한 청크로 판단되어 해당 문서가 저장되지 않음")
            return False
        
        
    def delete_vector_db_by_doc_id(self, filename):
        """
        주어진 문서 ID에 해당하는 벡터 임베딩을 삭제
        """
        # 벡터 데이터베이스에서 모든 문서 가져오기
        db_data = self.vector_store._collection.get(include=["metadatas"])
        ids = db_data['ids']
        metadatas = db_data['metadatas']
        ids_to_delete = [id for id, metadata in zip(ids, metadatas) if metadata.get('source') == filename]
    
        if ids_to_delete:
            self.vector_store._collection.delete(ids=ids_to_delete)
            print(f"[벡터 DB 삭제] 문서 ID [{filename}]의 임베딩을 벡터 DB에서 삭제했습니다.")
        else:
            print(f"[벡터 DB 삭제 실패] 문서 ID [{filename}]에 대한 임베딩을 찾을 수 없습니다.")
            

In [4]:
pipeline = Datapipeline()

## 1. Data Pipeline에 데이터 업로드하기 

### 먼저 빈 VectorDB를 확인해 봅니다. 

In [5]:
vectorDB = pipeline.init_vectorDB()                     # vectorDB가 새로 생겼으면, default 위치에 vectorDB를 가져오고 
# persist_dir = "./database"                            # 특정 vectorDB를 가져오고 싶다면 특정 폴더 위치를 지정해 줍니다.
# vectorDB = pipeline.init_vectorDB(persist_dir)

In [6]:
database = vectorDB.get()

database
# 비어 있는것을 확인할 수 있습니다.

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

### pipeline에서 수집한 데이터를 update_vector_db 함수를 통해 VectorDB에 넣어봅니다.

In [7]:
upload_file = 'documents\[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf'

In [8]:
with open(upload_file, 'rb') as file:                               # 비슷하거나 같은 내용이더라도, 새롭게 업데이트된 내용이라 넣어주고 싶다면 force=True로 
    print(file.name)
    pipeline.update_vector_db(file, force=True, chunk_size=200)     # 이렇게 정확히 명시해줘도 되고 
    # pipeline.update_vector_db(file)                               # default값으로 넣어줘도 됩니다. 

documents\[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf


100%|██████████| 4/4 [00:00<00:00, 4000.29it/s]


Added 46 new documents to the vector store


In [9]:
with open(upload_file, 'rb') as file:
    pipeline.update_vector_db(file, force=False, chunk_size=200)     # False로 하는 경우, 중복된 내용의 파일이 들어가는것을 방지해줍니다.

 25%|██▌       | 1/4 [00:00<00:00,  3.29it/s]

기존 DB에 유사한 청크가 있음으로 판단되어 추가되지 않음  - 0.07657945993175419


 50%|█████     | 2/4 [00:00<00:00,  2.96it/s]

기존 DB에 유사한 청크가 있음으로 판단되어 추가되지 않음  - 0.07221324124943584


 75%|███████▌  | 3/4 [00:00<00:00,  3.06it/s]

기존 DB에 유사한 청크가 있음으로 판단되어 추가되지 않음  - 0.06632672037143994


100%|██████████| 4/4 [00:01<00:00,  3.00it/s]

기존 DB에 유사한 청크가 있음으로 판단되어 추가되지 않음  - 2.306911399130307e-06
모두 유사한 청크로 판단되어 해당 문서가 저장되지 않음


In [10]:
# vectorDB에 들어갔는지 확인하기
database = vectorDB.get()   # database = vectorDB._collection.get(include=["metadatas"])        # 이렇게 하면 documents 내용 빼고 ids랑 metadatas만 깔끔하게 가져와서 볼 수 있음

database
# 비어 있는것을 확인할 수 있습니다.

{'ids': ['03f8cb11-880b-48cd-8d54-ad0f1d482f57',
  '0906c8af-7287-49ae-a953-cbd3242bcc29',
  '0e657601-d1ff-4a09-aace-34773821a326',
  '1337e63b-b9ad-4bfe-b08b-d137835d510e',
  '168c4ae4-7bfc-487d-b2af-7bc7a6bee42e',
  '189110b4-03b9-42bf-b232-a20665d14a70',
  '1c5fa918-3bee-4771-9c71-3c2bd13aaab3',
  '1ee0ec01-2cf4-4484-9b4b-fc8207dac555',
  '21521398-66ce-4b00-82da-432d849ad3b0',
  '253919f2-6912-4764-be2c-8b414604ff56',
  '29c1ef3f-85aa-4f5a-bfe8-8cca916de246',
  '2e1f868a-bb1d-4087-a68c-fd328ea033c5',
  '33a92598-50e1-4851-8433-3a1d704576c1',
  '34bac4e7-f37d-461a-a2b0-b787ad0a3615',
  '3acba865-6153-4477-a96a-7886900950c6',
  '45887f31-f4cc-4a67-bee9-39dd76391d69',
  '45c7598a-9c99-4a00-9188-9ed0b60b658e',
  '4827ae73-1e38-46b3-afc7-d02f8566e0d8',
  '58d6223f-6667-44d7-85fa-813adecdf7ca',
  '602e69af-7991-46f6-8318-e547cbc691eb',
  '63685e7d-2f19-42a1-98a5-409dcd81ac53',
  '66c0254d-bcb9-4f64-984e-569571dce17c',
  '707dad58-a465-4c44-8e86-5a85a3c1cc09',
  '85594ede-cb3d-4a4a-b3ee-

## 원하는 pdf파일의 내용들 삭제하기

In [11]:
pdf_file = '[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf'

In [12]:
pipeline.delete_vector_db_by_doc_id(pdf_file)

[벡터 DB 삭제] 문서 ID [[정책][제약산업정보포털][2019.04.08]인도 통관 및 운송.pdf]의 임베딩을 벡터 DB에서 삭제했습니다.


In [13]:
all_documents = vectorDB._collection.get(include=["metadatas"])

all_documents
# 삭제 된거 확인 

{'ids': [],
 'embeddings': None,
 'metadatas': [],
 'documents': None,
 'uris': None,
 'data': None,
 'included': ['metadatas']}

### 쿼리를 통해 VectorDB에 유사도 기준 유사도가 높은 내용을 가져와 본다.

In [14]:
# 다시 파일 넣어 VectorDB 채우고 
with open(upload_file, 'rb') as file:                               # 비슷하거나 같은 내용이더라도, 새롭게 업데이트된 내용이라 넣어주고 싶다면 force=True로 
    pipeline.update_vector_db(file, force=True, chunk_size=200)     # 이렇게 정확히 명시해줘도 되고 
    # pipeline.update_vector_db(file)                               # default값으로 넣어줘도 됩니다. 

upload_file = 'documents\[한인도관계][주인도대사관][2024.04.26]인도경제소식지.pdf'
with open(upload_file, 'rb') as file:                               # 비슷하거나 같은 내용이더라도, 새롭게 업데이트된 내용이라 넣어주고 싶다면 force=True로 
    pipeline.update_vector_db(file)                               # default값으로 넣어줘도 됩니다. 

100%|██████████| 4/4 [00:00<?, ?it/s]


Added 25 new documents to the vector store


100%|██████████| 27/27 [00:00<00:00, 26226.54it/s]


Added 126 new documents to the vector store


In [12]:
# 문서 조회1
query = '인도 통관 및 운송'   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = vectorDB.similarity_search(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(len(result))
print(result[0].page_content)

print('-'*50)
print()
for i, doc in enumerate(result):
    print(i)
    print(f"문서 내용: {doc.page_content}") # 문서 내용 표시
    print('---'*10)

3
5. 통관 및 운송
 
 
가. 통관제도
  
 
통관 유형별 절차
 
1) 정식통관 
 
인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 
--------------------------------------------------

0
문서 내용: 5. 통관 및 운송
 
 
가. 통관제도
  
 
통관 유형별 절차
 
1) 정식통관 
 
인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 
------------------------------
1
문서 내용: 국가정보(무역-통관 및 운송) 항목은 "공공누리 1유형 출처표시" 조건에 따라
이용 할 수 있습니다.
------------------------------
2
문서 내용: 후부터 체화료가 부과한다. 
 
2) 임시 통관 
 
인도에 들여온 품목을 사용하지 않고 24개월 이내에 다시 반출할 목적이 있는 경우 임시통관이 이루어지게 된다. 임시통관을 위해
------------------------------


### MMR 검색 

In [13]:
mmr_docs = vectorDB.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)

4
5. 통관 및 운송
 
 
가. 통관제도
  
 
통관 유형별 절차
 
1) 정식통관 
 
인도에서 일반적인 경우 통관에 소요되는 시간은 행정상 운송 수입의 경우 3~4 근무일, 


In [14]:
# 다음은 MMR 검색 결과 중 가장 낮은 순위(이 경우 4번째)의 문서의 내용을 출력합니다.
print(mmr_docs[-1].page_content)

경우 약식통관이 가능하다. 이는 전자상거래의 물품에도 적용된다. 인도는 2,000루피 이하의 품목을
3가지 분류로 나누어 HS코드를 적용하고 있으며, 그 분류는 다음과 같다.   


In [15]:
question = "인도 통관 및 운송에 대해서 알려줘."

results = vectorDB.similarity_search_with_score(question, k=1)

results[0][1]

0.11479044734842947

In [16]:
question = "럭키비키가 무엇인지 알려줘."

results = vectorDB.similarity_search_with_score(question, k=3)

results[0][1]

0.2179300793545511